In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from collections import OrderedDict
import pandas as pd
import re
from tqdm import tqdm

from io import StringIO
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [2]:
# python inference_qa_with_nli_validation.py 
# -d squadid 
# -msc indonli_mnli_squadid-nli 
# -tq entailment_or_neutral
# -msi 1 
# -va 2 
# -th 0.50 

In [3]:
prefix = "python inference_qa_with_nli_validation.py"

In [4]:
def take_all_url_nli_validator(data):
    
    msc_arr = ["indonli", "indonli_mnli", f"indonli_mnli_{data}-nli"]
    tq_arr = ['entailment_only', 'entailment_or_neutral']
    msi_arr = [1, 2, 3]
    var_arr = [1, 2, 3]
    th_arr = [0.25, 0.5, 0.75]

    url_arr = []

    for msc in msc_arr:
        for tq in tq_arr:
            for msi in msi_arr:
                for var in var_arr:
                    for th in th_arr:
                        
                        if msc == "indonli" or msc == "indonli_mnli":
                            var = 1
                            th = 0.0

                        if var == 1:
                            th = 0.0
                            
                        if (var == 3) and (msi == 1):
                            continue
                        
                        url_arr.append(f"{prefix} -d {data} -msc {msc} -tq {tq} -msi {msi} -va {var} -th {th}")
    
    url_arr = list(OrderedDict.fromkeys(url_arr))
    print("Amount of url:", len(url_arr))
    return url_arr

In [5]:
take_all_url_nli_validator("idkmrc")

Amount of url: 48


['python inference_qa_with_nli_validation.py -d idkmrc -msc indonli -tq entailment_only -msi 1 -va 1 -th 0.0',
 'python inference_qa_with_nli_validation.py -d idkmrc -msc indonli -tq entailment_only -msi 2 -va 1 -th 0.0',
 'python inference_qa_with_nli_validation.py -d idkmrc -msc indonli -tq entailment_only -msi 3 -va 1 -th 0.0',
 'python inference_qa_with_nli_validation.py -d idkmrc -msc indonli -tq entailment_or_neutral -msi 1 -va 1 -th 0.0',
 'python inference_qa_with_nli_validation.py -d idkmrc -msc indonli -tq entailment_or_neutral -msi 2 -va 1 -th 0.0',
 'python inference_qa_with_nli_validation.py -d idkmrc -msc indonli -tq entailment_or_neutral -msi 3 -va 1 -th 0.0',
 'python inference_qa_with_nli_validation.py -d idkmrc -msc indonli_mnli -tq entailment_only -msi 1 -va 1 -th 0.0',
 'python inference_qa_with_nli_validation.py -d idkmrc -msc indonli_mnli -tq entailment_only -msi 2 -va 1 -th 0.0',
 'python inference_qa_with_nli_validation.py -d idkmrc -msc indonli_mnli -tq entailm

In [6]:
def take_dataframe(url): 
    
    response = requests.get(url)
    
    if response.status_code == 200:
        csv_data = StringIO(response.text)
        df = pd.read_csv(csv_data, index_col=0)
        df = df[['Prediction Answer', 'Rec. Pred Answer', 'Gold Answer', 'Properties']]
        df.fillna('', inplace=True)
    
    else:
        print("Failed to download CSV")
    
    return df

def take_accepted_and_rejected_qa_with_nli(url):
    
    df = take_dataframe(url)
    
    right_pred_model_qa_and_accepted_by_nli_ctr = 0
    right_pred_model_qa_and_rejected_by_nli_ctr = 0
    wrong_pred_model_qa_and_accepted_by_nli_ctr = 0
    wrong_pred_model_qa_and_rejected_by_nli_ctr = 0
    
    for i in range(len(df)):
        
        gold_answer = df['Gold Answer'][i]
        qa_generated_answer = eval(df['Rec. Pred Answer'][i])[0]
        nli_validated_answer = df['Prediction Answer'][i]
    
        # Jawaban dari model qa yg benar & di-accept sama model NLI
        right_pred_model_qa_and_accepted_by_nli = (gold_answer == qa_generated_answer) and \
            (qa_generated_answer == nli_validated_answer)

        # Jawaban dari model qa yg benar & di-reject sama model NLI
        right_pred_model_qa_and_rejected_by_nli = (gold_answer == qa_generated_answer) and \
            (qa_generated_answer != nli_validated_answer)

        # Jawaban dari model qa yg salah & di-accept sama model NLI
        wrong_pred_model_qa_and_accepted_by_nli = (gold_answer != qa_generated_answer) and \
            (qa_generated_answer == nli_validated_answer)

        # Jawaban dari model qa yg salah & di-reject sama model NLI
        wrong_pred_model_qa_and_rejected_by_nli = (gold_answer != qa_generated_answer) and \
            (qa_generated_answer != nli_validated_answer)
        
        if right_pred_model_qa_and_accepted_by_nli:
            right_pred_model_qa_and_accepted_by_nli_ctr += 1
        elif right_pred_model_qa_and_rejected_by_nli:
            right_pred_model_qa_and_rejected_by_nli_ctr += 1
        elif wrong_pred_model_qa_and_accepted_by_nli:
            wrong_pred_model_qa_and_accepted_by_nli_ctr += 1
        elif wrong_pred_model_qa_and_rejected_by_nli:
            wrong_pred_model_qa_and_rejected_by_nli_ctr += 1
    
    return right_pred_model_qa_and_accepted_by_nli_ctr, right_pred_model_qa_and_rejected_by_nli_ctr, \
            wrong_pred_model_qa_and_accepted_by_nli_ctr, wrong_pred_model_qa_and_rejected_by_nli_ctr

In [8]:
#indonli_mnli_idkmrc-nli	idkmrc	1	4	1	2	0,75	75,94787	84,16834	639	1	198	6

In [9]:
url = "https://huggingface.co/muhammadravi251001/fine-tuned-FilteringNLI-indonli_mnli_idkmrc-nli-idkmrc-TQ1-TS4-MS1-VA2-TH0.75/raw/main/results/output/output_df.csv"

take_accepted_and_rejected_qa_with_nli(url)

(395, 245, 56, 148)

In [18]:
def count_unanswerable_pred_and_gold_answer(url):
    
    df = take_dataframe(url)
    
    unanswerable_pred_ctr = 0
    unanswerable_pred_and_gold_answer_ctr = 0
    
    for i in tqdm(range(len(df))):
        
        pred_answer = df['Prediction Answer'][i]
        gold_answer = df['Gold Answer'][i]
        
        if pred_answer == '':
            unanswerable_pred_ctr += 1
        
        if pred_answer == '' and gold_answer != '':
            unanswerable_pred_and_gold_answer_ctr += 1
        
    return unanswerable_pred_ctr, unanswerable_pred_and_gold_answer_ctr

In [19]:
x = count_unanswerable_pred_and_gold_answer(url)

print(x)

100%|██████████████████████████████████████████████████████████████████████████████| 844/844 [00:00<00:00, 98486.33it/s]

(826, 415)


In [21]:
def find_sample_of_unanswerable_pred_and_gold_answer(url):
    
    df = take_dataframe(url)
    
    unanswerable_pred_and_gold_answer_rows = []

    for i in range(len(df)):
        
        pred_answer = df['Prediction Answer'][i]
        gold_answer = df['Gold Answer'][i]
        
        unanswerable_pred_and_gold_answer = (pred_answer == '') and (gold_answer != '')
        
        if unanswerable_pred_and_gold_answer:
            unanswerable_pred_and_gold_answer_rows.append(df.iloc[i])

    unanswerable_pred_and_gold_answer_df = pd.DataFrame(unanswerable_pred_and_gold_answer_rows).reset_index(drop=True)

    return unanswerable_pred_and_gold_answer_df

In [22]:
df = find_sample_of_unanswerable_pred_and_gold_answer(url)
df

,Prediction Answer,Rec. Pred Answer,Gold Answer,Properties
0,,['1959'],1959,(Blank) answer saved from variation 2 without ...
1,,['komponen dari sebuah nama pribadi yang berda...,Komomene sebuah nama yang berdasarkan pada nam...,(Blank) answer saved from variation 2 without ...
2,,['Gubernur DKI Jakarta'],Gubernur DKI Jakarta yang menjabat sejak 19 No...,(Blank) answer saved from variation 2 without ...
3,,['Ir. Basuki Tjahaja Purnama'],Basuki Tjahaja Purnama,(Blank) answer saved from variation 2 without ...
4,,['Januari 414'],Januari 414,(Blank) answer saved from variation 2 without ...
...,...,...,...,...
410,,['1939'],1939,(Blank) answer saved from variation 2 without ...
411,,['28997m2'],29095m²,(Blank) answer saved from variation 2 without ...
412,,['3 ton'],3 ton,(Blank) answer saved from variation 2 without ...
413,,['131.3 mil persegi'],"Menurut Biro Sensus Amerika Serikat, kota ini ...",(Blank) answer saved from variation 2 without ...


In [27]:
def count_wa_and_wr(df):
    
    wa_ctr = 0
    wr_ctr = 0
    
    for i in range(len(df)):
        
        gold_answer = df['Gold Answer'][i]
        qa_generated_answer = eval(df['Rec. Pred Answer'][i])[0]
        nli_validated_answer = df['Prediction Answer'][i]
        
        ra = (gold_answer == qa_generated_answer) and (qa_generated_answer == nli_validated_answer)
        rr = (gold_answer == qa_generated_answer) and (qa_generated_answer != nli_validated_answer)
        wa = (gold_answer != qa_generated_answer) and (qa_generated_answer == nli_validated_answer)
        wr = (gold_answer != qa_generated_answer) and (qa_generated_answer != nli_validated_answer)
        
        if wa:
            wa_ctr += 1
        if wr:
            wr_ctr += 1
    
    assert (wa_ctr + wr_ctr) == len(df)
    return wa_ctr, wr_ctr

In [28]:
df = find_sample_of_unanswerable_pred_and_gold_answer(url)
x = count_wa_and_wr(df)
x

AssertionError: 